# Real-world network

In [4]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [5]:
g = nx.read_gml("./RealNetworks/euroroad.gml")

In [6]:
print(nx.info(g))

Graph with 1039 nodes and 1305 edges


<ipython-input-6-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [7]:
nx.is_connected(g)

True

In [8]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [9]:
from myTruss import mappingAndRelabeling

In [10]:
g_relabled = mappingAndRelabeling(g)

In [11]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 1039 nodes and 1305 edges
Graph with 1039 nodes and 1305 edges


<ipython-input-11-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-11-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [12]:
g = g_relabled

# Infer community structure with Louvain

In [13]:
nx.__version__

'2.8.4'

In [14]:
import pandas as pd
from os.path  import exists
import infomap as im

In [15]:
im.__version__

'2.3.0'

In [16]:
G = g

In [17]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [18]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [19]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [20]:
results_file = "./results/euroroad-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/euroroad-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/euroroad-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/euroroad-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [21]:
len(mec_ranks)

1039

In [22]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[448,
 1,
 53,
 380,
 168,
 158,
 62,
 376,
 285,
 286,
 229,
 405,
 453,
 681,
 284,
 423,
 174,
 166,
 181,
 124,
 102,
 104,
 5,
 329,
 300,
 679,
 8,
 61,
 7,
 66,
 6,
 69,
 402,
 404,
 446,
 279,
 647,
 233,
 375,
 692,
 427,
 461,
 730,
 165,
 71,
 236,
 148,
 209,
 156,
 277,
 108,
 87,
 86,
 542,
 628,
 313,
 320,
 391,
 395,
 316,
 47,
 94,
 605,
 13,
 19,
 3,
 10,
 330,
 332,
 240,
 304,
 878,
 642,
 78,
 80,
 593,
 29,
 11,
 30,
 59,
 50,
 58,
 56,
 65,
 513,
 847,
 276,
 408,
 406,
 849,
 374,
 457,
 586,
 587,
 469,
 368,
 293,
 683,
 650,
 282,
 288,
 616,
 234,
 235,
 378,
 418,
 654,
 697,
 421,
 413,
 693,
 429,
 455,
 458,
 865,
 621,
 622,
 624,
 674,
 175,
 167,
 239,
 237,
 115,
 114,
 140,
 147,
 142,
 207,
 145,
 171,
 220,
 610,
 612,
 224,
 129,
 88,
 193,
 132,
 91,
 89,
 537,
 556,
 627,
 525,
 718,
 505,
 388,
 393,
 857,
 157,
 93,
 95,
 101,
 103,
 346,
 572,
 358,
 362,
 364,
 15,
 21,
 22,
 32,
 24,
 14,
 493,
 553,
 564,
 555,
 547,
 534,
 259,
 328,
 3

In [23]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [24]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [25]:
dict_map_eq_centrality

{448: 1039,
 1: 1038,
 53: 1037,
 380: 1036,
 168: 1035,
 158: 1034,
 62: 1033,
 376: 1032,
 285: 1031,
 286: 1030,
 229: 1029,
 405: 1028,
 453: 1027,
 681: 1026,
 284: 1025,
 423: 1024,
 174: 1023,
 166: 1022,
 181: 1021,
 124: 1020,
 102: 1019,
 104: 1018,
 5: 1017,
 329: 1016,
 300: 1015,
 679: 1014,
 8: 1013,
 61: 1012,
 7: 1011,
 66: 1010,
 6: 1009,
 69: 1008,
 402: 1007,
 404: 1006,
 446: 1005,
 279: 1004,
 647: 1003,
 233: 1002,
 375: 1001,
 692: 1000,
 427: 999,
 461: 998,
 730: 997,
 165: 996,
 71: 995,
 236: 994,
 148: 993,
 209: 992,
 156: 991,
 277: 990,
 108: 989,
 87: 988,
 86: 987,
 542: 986,
 628: 985,
 313: 984,
 320: 983,
 391: 982,
 395: 981,
 316: 980,
 47: 979,
 94: 978,
 605: 977,
 13: 976,
 19: 975,
 3: 974,
 10: 973,
 330: 972,
 332: 971,
 240: 970,
 304: 969,
 878: 968,
 642: 967,
 78: 966,
 80: 965,
 593: 964,
 29: 963,
 11: 962,
 30: 961,
 59: 960,
 50: 959,
 58: 958,
 56: 957,
 65: 956,
 513: 955,
 847: 954,
 276: 953,
 408: 952,
 406: 951,
 849: 950,
 374:

In [26]:
# Write Centrality
with open('./MapEquationCentralityResults/Infra-Euroroad/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [27]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Infra-Euroroad/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [28]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True